# Co-factor swap

<!-- The balance of co-factors within the cells is important 
In the implementated pathway of progesterone nine NADP and one NAD molecules 
In order for the cell to produce one pregesterone molecule through the implementated pathway nine NADPH molecules and one NAD $^+$ molecules are needed. 
 -->

The banlance of co-factors within a cell is important to obtain a high yield of a given product (REF!!!!!!!!!!!!!!). Nine NADPH molecules and one NAD $^+$ molecules are needed in order for a cell to produce one progesterone molecule through the implementated pathway. Therefore, by increasing the concentration of avaliable NADPH/NADP $^+$ and NADH/NAD $^+$ co-factors in the the cell, it should be theoretically possible to increase of the yield of progesterone. 
The XXXXXXXXXXX algorithm was used to obtain possible targets

In [1]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [2]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [3]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [4]:
# Finding the optimal fluxes through each reaction when the objective is biomass production 
Fluxes = model.optimize()
Fluxes

,fluxes,reduced_costs
CITtcp,0.00000,0.000000
13BGH,0.00000,-0.047516
13BGHe,0.00000,0.000000
13GS,0.32667,0.000000
16GS,0.00000,0.000000
...,...,...
R03353,0.00000,0.000000
R07215,0.00000,0.000000
R04804,0.00000,0.000000
SK_methylpentanal_c,0.00000,0.000000


In [5]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [6]:
# Setting the progesterone yield from glucose as substrate
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [7]:
#optswap

In [8]:
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap_1 = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["nadp_c","nadph_c"]),objective_function=py)

swap_1

In [9]:
# Identify the reactions that when swapped to another co-factor improve the production of progesterone
solution = swap_1.run(max_size = 10)

Starting optimization at Mon, 21 Nov 2022 11:47:23


HBox()

Finished after 00:01:32


In [10]:
solution

,index,targets,fitness
0,0,"(GAPD,)",0.342857
1,3,"(ALCD2x_copy1,)",0.323077
2,4,"(ALCD2x_copy2,)",0.323077
3,7,"(MDH, R05703)",0.257668
4,8,"(R05703, HMGCOAR)",0.257668
5,9,"(HMGCOAR, C3STDH1)",0.257668
6,10,"(MDH,)",0.253012
7,12,"(HMGCOAR,)",0.253012
8,13,"(ECYP11A1, R07498, R07215)",0.235955
9,14,"(ECYP11A1,)",0.228261


In [11]:
solution.model

Name,iMM904_progesterone
Memory address,0x07fb1f1e5e820
Number of metabolites,1234
Number of reactions,1587
Number of groups,0
Objective expression,1.0*R02216 - 1.0*R02216_reverse_2ed70
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [12]:
reactions=solution.data_frame.targets
reactions

0                        (GAPD,)
1                (ALCD2x_copy1,)
2                (ALCD2x_copy2,)
3                  (MDH, R05703)
4              (R05703, HMGCOAR)
5             (HMGCOAR, C3STDH1)
6                         (MDH,)
7                     (HMGCOAR,)
8     (ECYP11A1, R07498, R07215)
9                    (ECYP11A1,)
10                    (C4STMO1,)
11              (R05703, R07215)
12                    (C3STDH1,)
13                     (R07215,)
14                       (SQLS,)
15                     (R02216,)
16                    (C3STKR1,)
17                     (R07498,)
18                     (C14STR,)
19                     (R01456,)
20                    (C3STKR2,)
21                    (C4STMO2,)
Name: targets, dtype: object

In [13]:
reactions[2][0]

'ALCD2x_copy2'

In [14]:
model.reactions.get_by_id(reactions[2][0])

Reaction identifier,ALCD2x_copy2
Name,Alcohol dehydrogenase (ethanol)
Memory address,0x07fb17a969f70
Stoichiometry,etoh_c + nad_c <=> acald_c + h_c + nadh_c Ethanol + Nicotinamide adenine dinucleotide <=> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YDL168W
Lower bound,-999999.0
Upper bound,999999.0


In [15]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
Ethanol + Nicotinamide adenine dinucleotide ==> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
Ethanol + Nicotinamide adenine dinucleotide ==> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
L-Malate + Nicotinamide adenine dinucleotide ==> H+ + Oxaloacetate + Nicotinamide adenine dinucleotide - reduced

5alpha-Cholesta-7_24-dien-3beta-ol + H+ + Nicotinamide adenine dinucleotide phosphate - reduced ==> lathosterol + Nicotinamide adenine dinucleotide phosphate

Reactions:
5alpha-Cholesta-7_24-dien-3beta-ol + H+ + Nicotinamide adenine dinucleotide phosphate - reduced ==> lathosterol + Nicotinamide adenine dinucleotide phosphate

Coenzyme A +  R  Mevalonate C6H11O4 + Nicotinamide adenine dinucleotide phosphate ==> H+ + Hydroxymethylglutaryl CoA C27H39N

In [16]:
py